In [78]:
import pandas as pd
from google.colab import drive
drive.mount('/drive')
%cd /drive/MyDrive/data/pipotron/
%pwd
%ls -al

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
/drive/MyDrive/data/pipotron
total 52
drwx------ 2 root root  4096 Mar 16 10:48 donnees/
drwx------ 8 root root  4096 Mar 16 10:40 .git/
-rw------- 1 root root  1799 Mar 16 10:40 .gitignore
-rw------- 1 root root  3522 Mar 16 10:41 git.ipynb
drwx------ 3 root root  4096 Mar 16 10:46 models/
-rw------- 1 root root 33797 Mar 16 11:00 pipotron_entrainement.ipynb
-rw------- 1 root root    79 Mar 16 10:40 README.md


In [88]:
# Chargement des données :

from_full_table = False
reviews_filename = "donnees/review_only.txt"

if from_full_table:
  # Chargement du fichier complet, issu du scraping du site larvf.com (données accessibles par abonnement, non publiques):
  DF_rvf = pd.read_csv("/drive/MyDrive/donnees/larvf_2020-11-24.csv")
  print("Nombre de revues textuelles disponibles :", len(DF_rvf)-DF_rvf['review_text'].isna().sum())

  # On sélectionne la colonne qui nous intéresse en retirant les valeurs manquantes, et on fait un mélange aléatoire: 
  S_reviews = DF_rvf['review_text'][~DF_rvf['review_text'].isna()].sample(frac=1, random_state=5406).reset_index(drop=True)
  display(S_reviews.head(4))
  
  # On enregistre ces données pour ce projet:
  S_reviews.to_csv(reviews_filename, header=False, index=False)

else:
  S_reviews = pd.read_csv(reviews_filename, header=None).iloc[:, 0]
  display(S_reviews.head(4))


0    Cette cuvée a gagné en élégance et en délicate...
1    Bonne trame acide. Légères notes animales à l’...
2    Réservé, à la fois pur et nerveux, il s'appuie...
3    Un assemblage de meunier et chardonnay, pas d’...
Name: 0, dtype: object

In [89]:
# Préparation des données:
import re

# On ajoute un nouveau mot (token) qui a vocation à servir de déclencheur pour générer un nouveau commentaire de dégustation.
# On supprime également les informations de date du commentaire, parfois présentes.
# Enfin on ajoute un token de fin de séquence pour inciter le modèle à produire des commentaires de taille raisonnable.
L_first_tokens = ["<|review|>"]
L_reviews = [L_first_tokens[0] + " " + re.sub(r"\([^\(\)]*\.[0-9][0-9][0-9][0-9]\)", "", review.strip()) for review in list(S_reviews)]
L_reviews = [review + " <|end|>" for review in L_reviews]

# Affichons le résultat de cette préparation sur quelques lignes:
for i in range(27,32):
  print(L_reviews[i])

<|review|> On sent des vignes qui ont du fond dans un vin mûr et structuré, qui reste simple en finale. <|end|>
<|review|> Un véritable jus de caillou croquant et tonique, un vin plein de répartie, singulier dans l'éclat très ferme de son fruit tendu, d'une rare intensité désaltérante et juteuse. <|end|>
<|review|> Il s’exprime sur le fruit noir. Il mêle virilité et élégance, avec une forte empreinte du terroir. <|end|>
<|review|> Derrière une fine réduction, il livre une note d’épices et de garrigue. Il lui faut un peu d’air pour libérer son fruit. Il offre une très belle qualité du fruit, du relief avec une jolie assise tannique mais sans dureté. Long en bouche, il possède un beau potentiel de garde et d’évolution. <|end|>
<|review|> Les vendanges ont été plus tardives qu’à Haut-Brion, mais le vin conserve une réelle fraîcheur et un fruit sapide. Beaucoup de crémeux avec une fine sucrosité et un boisé doux (60 % de bois neuf). Bouche de grand équilibre avec une saveur saline en final

In [90]:
# On vérifie le nombre de lignes:
print(len(L_reviews))
assert len(L_reviews)==len(DF_rvf)-DF_rvf['review_text'].isna().sum()

30667


In [96]:
# On va s'appuyer sur un modèle de générateur GPT-2 (merci OpenAI) pré-entraîné sur la langue française (merci Antoine Louis),
# et on utilise la bibliothèque transformers (merci à HuggingFace): 
!pip install transformers
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

# On charge le modèle pré-entraîné et son tokenizer associé:
model_name = "antoiloui/belgpt2"
model = TFGPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
nb_added_tokens = 0

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at antoiloui/belgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [97]:
# On ajoute des tokens de déclenchement et de padding au tokenizer, et on prépare le modèle à recevoir ces nouveaux tokens:
nb_added_tokens += tokenizer.add_special_tokens({'pad_token': "<|pad|>", 'eos_token':"<|end|>"})
if len(L_first_tokens)>0:
    nb_added_tokens += tokenizer.add_special_tokens({'additional_special_tokens': L_first_tokens})
print(nb_added_tokens, "token(s) ajoutés")

_ = model.resize_token_embeddings(tokenizer.vocab_size + nb_added_tokens)

3 token(s) ajoutés


In [98]:
# On vérifie la bonne définition des tokens de padding et de fin de séquence :
print(tokenizer.pad_token, ":", tokenizer.pad_token_id)
assert tokenizer.pad_token_id!=None
print(tokenizer.eos_token, ":", tokenizer.eos_token_id)
assert tokenizer.eos_token_id!=None

<|pad|> : 50257
<|end|> : 50258


In [99]:
# On vérifie qu'un commentaire de vin est inchangé après tokenisation puis détokenisation:
i = 4449
print(L_reviews[i])
print(tokenizer.decode(tokenizer.encode(L_reviews[i])))
assert L_reviews[i] == tokenizer.decode(tokenizer.encode(L_reviews[i]))

<|review|> La précision de la matière entretient la force qui se dégage de ce vin. Amples, les tanins encore denses restent intégrés à la matière et soutiennent un fruité généreux. Les notes de cacao portent l'allonge avec puissance. <|end|>
<|review|> La précision de la matière entretient la force qui se dégage de ce vin. Amples, les tanins encore denses restent intégrés à la matière et soutiennent un fruité généreux. Les notes de cacao portent l'allonge avec puissance. <|end|>


In [100]:
# On génère un commentaire aléatoire, avant entraînement spécifique (fine tuning) donc non exploitable à ce stade:

# TO DO: optimiser si besoin (?) les paramètres de génération aléatoire 
# cf. https://blog.fastforwardlabs.com/2019/05/29/open-ended-text-generation.html

def pipote(max_length=200, skip_special_tokens=True):
  input = tokenizer.encode(L_first_tokens[0], return_tensors='tf')
  output = model.generate(
      input_ids=input,
      max_length=max_length,
      do_sample=True,
      pad_token_id=tokenizer.pad_token_id 
  )
  return tokenizer.decode(output[0], skip_special_tokens=skip_special_tokens)

print(pipote())

ages, les murs ont été arrachés, des vitres ont été brisées... ", se plaint le propriétaire. Depuis l' année 2010, elles ont en effet été multipliées par environ 10 et elles sont prêtes à atteindre 5,3 milliards d' euros d' ici la fin du premier semestre 2011. De plus en plus de parents sont touchés par le problème " Je ne suis pas d' accord avec toutes les décisions de ce genre, je refuse de mettre de côté quelque chose du programme. Mais il ne fallait pas s' attendre à ce qu' il le dise : il avait raison. Nous sommes une démocratie représentative et cette démocratie est le socle sur lequel elle repose. Les deux hommes ont été mis en examen pour " tentative d' assassinat " et placés en détention provisoire, a précisé la même source. Dans un discours pour mettre fin aux hostilités dans la région, le président français, accompagné de la chancelière allemande, Angela Merkel, avait réaffirmé le droit du gouvernement malien à la sécurité des militaires français,


In [ ]:
# On prépare les données d'entraînement du générateur:
# Le générateur prédit le mot suivant pour chaque mot (ou plutôt token, en l'occurrence wordpiece) du texte fourni en entrée.
# Donc l'ensemble d'apprentissage est constitué de tuples (x=entrée, y=sortie) où y est un décalage de x d'un token vers la droite.

# TO DO: train/test split

batch_size = 16
train_size = int(0.75*len(L_reviews)/batch_size)*batch_size
print("Taille des batches :", batch_size)
print("Nombre de lignes dans la base d'apprentissage :", train_size)

encodings = tokenizer(L_reviews, padding=True, return_tensors='tf')
input_ids = encodings.input_ids
attention_mask = encodings.attention_mask
x = {'input_ids': input_ids[:, :-1], 'attention_mask': attention_mask[:, :-1]}
y = input_ids[:, 1:]
assert x['input_ids'].shape == y.shape
assert x['attention_mask'].shape == y.shape
full_dataset = tf.data.Dataset.from_tensor_slices((x, y))
train_dataset = full_dataset.take(train_size).batch(batch_size).repeat()
test_dataset = full_dataset.skip(train_size).batch(batch_size)

Taille des batches : 16
Nombre de lignes dans la base d'apprentissage : 22992


In [ ]:
# On "encapsule" le modèle initial dans un modèle qui ne renvoie que les "logit" des probabilités des mots,
# ce à quoi on peut appliquer la fonction de coût classique (entropie croisée)
class Pipotron(tf.keras.Model):
  def __init__(self, model):
    super().__init__(name="Pipotron")
    self.gpt2 = model
  def __call__(self, input, training=False):    
    y = self.gpt2(input, training=training)
    return y.logits

pipotron = Pipotron(model)

In [ ]:
# On contrôle les formats d'entrée et de sortie du modèle (à ce stade il est déboussolé par notre token de déclenchement):
for features, labels in train_dataset.take(1):
  i=4
  for k in features.keys():
    print(k, ":", features[k].shape)
  print(tokenizer.decode(features['input_ids'][i], skip_special_tokens=False))
  print("labels :", labels.shape)
  print(tokenizer.decode(labels[i], skip_special_tokens=False))
  z = pipotron(features)
  print("output :", z.shape)
  print(tokenizer.decode(tf.math.argmax(z[i], axis=-1), skip_special_tokens=False))

input_ids : (16, 250)
attention_mask : (16, 250)
<|review|> Un viognier fidèle à son bouquet exubérant, agréable et fluide. <|end|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pa

In [ ]:
# ENTRAINEMENT (fine tuning) DU MODELE :

# TO DO : appliquer un learning_rate dégressif

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
pipotron.compile(optimizer=optimizer, loss=loss)

# Pour voir la progression, on affiche le score à chaque 'round' (pseudo-epoch, qui est en fait une subdivision de la 'vraie' epoch
# si on considère qu'une vraie epoch consiste à passer toute la base d'entraînement une fois):
nb_epochs = 2
nb_steps_per_round = 400
nb_rounds = (nb_epochs*len(L_reviews))//(batch_size*nb_steps_per_round)
print("Nombre d'étapes (rounds) d'entraînement prévues :", nb_rounds)
H_history = pipotron.fit(train_dataset, validation_data=test_dataset, epochs=nb_rounds, steps_per_epoch=nb_steps_per_round)
#pipotron.fit(train_dataset, epochs=nb_rounds, steps_per_epoch=nb_steps_per_round)


Nombre d'étapes (rounds) d'entraînement prévues : 9
Epoch 1/9
400/400 [==============================] - 360s 871ms/step - loss: 1.1691 - val_loss: 0.5262
Epoch 2/9
400/400 [==============================] - 346s 866ms/step - loss: 0.5132 - val_loss: 0.4961
Epoch 3/9
400/400 [==============================] - 346s 867ms/step - loss: 0.4934 - val_loss: 0.4772
Epoch 4/9
400/400 [==============================] - 346s 867ms/step - loss: 0.4726 - val_loss: 0.4634
Epoch 5/9
400/400 [==============================] - 346s 866ms/step - loss: 0.4715 - val_loss: 0.4539
Epoch 6/9
400/400 [==============================] - 346s 866ms/step - loss: 0.4483 - val_loss: 0.4491
Epoch 7/9
400/400 [==============================] - 346s 866ms/step - loss: 0.4470 - val_loss: 0.4461
Epoch 8/9
400/400 [==============================] - 346s 866ms/step - loss: 0.4422 - val_loss: 0.4452
Epoch 9/9
400/400 [==============================] - 346s 866ms/step - loss: 0.4278 - val_loss: 0.4427


In [ ]:
# Sauvegarde du modèle:
import time

fullpath = "models/pipotron_" + str(int(time.time()/60))
model.save_pretrained(fullpath)
print("MODEL SAVED AT :", fullpath)

MODEL SAVED AT : /drive/MyDrive/saved_models/pipotron_26931474


In [101]:
# Rechargement du modèle, si besoin :

fullpath = "models/pipotron_26931474"
model = TFGPT2LMHeadModel.from_pretrained(fullpath)
pipotron = Pipotron(model)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at models/pipotron_26931474.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [102]:
# On affiche quelques exemples de commentaires générés par le modèle entraîné:
print(pipote())
print(pipote())
print(pipote())


Un rouge vineux et solaire, dans un style mûr et fin, qui doit se fondre. Jean,, se débarrasse du carcan, avec ses arômes de prune et ses accents oxydatifs.
Un vin demi-sec que le terroir a préservé, qui s'exprime dans une matière finement parfumée. Agréable! Baugain..
Le parfum et la texture très soyeuse des tanins du Corton Le Charmes-Chambertin ne trompent pas : les arômes sont poudrés et la bouche, bien mûre, intègre le millésime.
